In [1]:
import pandas as pd
import pandas.io.sql
import pymssql as sapsql 
import numpy as np
from IPython.display import display
from jinja2 import Environment, FileSystemLoader
import pdfkit 


env = Environment(loader=FileSystemLoader('/data/jupyter/'))
#template = env.get_template("template.html")

datacompany = ({'host':'192.168.1.13' , 'dbname':'IGU_LIVE','user':'sa','password':'B1admin'},)

datalist1=[]
 
for company in datacompany:
    msg_sql1= """
                 exec DBO.IGU_INVOICELIST '20200801','20200903','',''
               
                """

    conn = sapsql.connect(host=company["host"] , user=company["user"] , password=company["password"], database=company["dbname"] )
    cursor = conn.cursor()     
    data1 = pandas.io.sql.read_sql(msg_sql1,conn)
    datalist1.append(data1)

dfIn = pd.concat(datalist1)  
options = {
            'page-size': 'legal',
            'orientation': 'landscape',
            }
#html_out = template.render(template_vars)
textstring = ""
#test = df.pivot_table(index=["groupname",'cardname'],columns=["PymntGroup"],aggfunc=np.sum,  values=["Balance"],fill_value="0",margins=True)

#master = pd.DataFrame(df, columns = ['cardcode','cardname','groupname','U_AR_Person','LicTradNum','U_SlsEmpName','Balance','CreditLine','PymntGroup','U_locktimeout','U_IGU_LockBP',])

#for index,rows in data_ar.iterrows():
#    print(rows)

dfIn.info()
dfIn

#test

#pdfkit.from_string(html_out,path_PDFfile1,options=options,css=cssfile) 
#df.to_excel (r'invoicelist.xlsx', index=True, header=True, merge_cells=False)

env = Environment(loader=FileSystemLoader('/data/jupyter/'))
#template = env.get_template("template.html")

datacompany = ({'host':'192.168.1.13' , 'dbname':'IGU_LIVE','user':'sa','password':'B1admin'},)

datalist2=[]
 
for company in datacompany:
    msg_sql2= """
                   
                   select a.cardcode ,a.cardname ,d.groupname,a.U_AR_Person,c.U_SlsEmpName, a.Balance , a.CreditLine, a.U_locktimeout,
                            b.PymntGroup ,
                            b.ExtraDays days ,
                            b.ExtraMonth months

                    from OCRd a 
                        inner join octg b on a.GroupNum = b.GroupNum
                        inner join oslp c on a.slpcode = c.slpcode
                        inner join ocrg d on a.GroupCode = d.GroupCode
                    where a.cardtype='C'
                """

    conn = sapsql.connect(host=company["host"] , user=company["user"] , password=company["password"], database=company["dbname"] )
    cursor = conn.cursor()     
    data2 = pandas.io.sql.read_sql(msg_sql2,conn)
    datalist2.append(data2)

dfBP = pd.concat(datalist2)  
options = {
            'page-size': 'legal',
            'orientation': 'landscape',
            }
#html_out = template.render(template_vars)
textstring = ""
#test = df.pivot_table(index=["groupname",'cardname'],columns=["PymntGroup"],aggfunc=np.sum,  values=["Balance"],fill_value="0",margins=True)

dfBP = dfBP.fillna(0)                                                                                                                          

dfInBp = pandas.merge(dfIn, dfBP, how = 'left', on=["cardcode"])

dfInBp.info()

df = dfInBp[['docnum','NumAtCard','docdate','cardcode','ShipToCode','U_Kw_No','U_AR_Person','U_SlsEmpName','Balance','PymntGroup','days','U_locktimeout',]]
df.rename(columns = {'docnum':'No Invoice', 'NumAtCard' : 'No SO', 'docdate' : 'Invoice Date', 'cardcode' : 'Code','ShipToCode': 'Customer Name' ,'U_Kw_No': 'No kwitansi', 'U_AR_Person' : 'AR',
                    'U_SlsEmpName': 'Sales', 'Balance':'Nominal','PymntGroup':'Term Of Payment', 'days':'TOPdays','U_locktimeout':'LockTimeout'  }, inplace = True)

df = df[df.Nominal != 0]

df ['Date Today']=pd.Timestamp("today")
df ['Date Today']=df['Date Today'].map(lambda ts:ts.strftime("%Y-%m-%d"))

df['Invoice Date'] = pd.to_datetime(df['Invoice Date'])
df['Date Today'] = pd.to_datetime(df['Date Today'])

df['TOP Jatuh Tempo'] = df['Invoice Date'] + pd.to_timedelta(pd.np.ceil(df.TOPdays), unit="D")

df['H-7 JT days'] = (df['TOP Jatuh Tempo'] - df['Date Today'])

df['H-7 JT days'] = df['H-7 JT days'].dt.days

conditions = [
    (df['Nominal'] == 0),
    (df['H-7 JT days'] < 7) & (df['H-7 JT days'] >= 1) & (df['Nominal'] > 0),
    (df['H-7 JT days'] < 0) & (df['Nominal'] > 0),
    (df['H-7 JT days'] == 7) & (df['Nominal'] > 0),
    (df['H-7 JT days'] == 0) & (df['Nominal'] > 0),
       ]
choices = ['Lunas', 'Approaching JT', 'Overdue', "Today", "Jatuh Tempo"]

df['Alert H-7 JT'] = np.select(conditions, choices, default='')

df ['LockTimeout']=df['LockTimeout']*30

df['Date Locked'] = df['Invoice Date'] + pd.to_timedelta(pd.np.ceil(df.LockTimeout), unit="D")

df['H-7 Locked days'] = (df['Date Locked'] - df['Date Today'])

df['H-7 Locked days'] = df['H-7 Locked days'].dt.days

conditions = [
    (df['Nominal'] == 0),
    (df['H-7 Locked days'] < 7) & (df['H-7 Locked days'] >= 1) & (df['Nominal'] > 0),
    (df['H-7 Locked days'] == 0) & (df['Nominal'] > 0),
    (df['H-7 Locked days'] == 7) & (df['Nominal'] > 0),
       ]
choices = ['Lunas', 'Approaching locked', 'Locked', "Today" ]

df['Alert H-7 Locked'] = np.select(conditions, choices, default='')

df['H Locked days'] = df['H-7 Locked days']

conditions = [
    (df['H Locked days'] == 0),
          ]
choices = ['Locked', ]

df['Alert H-Locked'] = np.select(conditions, choices, default='')

#df['Beda'] = df['Beda'].dt.days.astype('int16')
df ['Date Today']=df['Date Today'].map(lambda ts:ts.strftime("%Y-%m-%d"))
df ['Invoice Date']=df['Invoice Date'].map(lambda ts:ts.strftime("%Y-%m-%d"))
df ['TOP Jatuh Tempo']=df['TOP Jatuh Tempo'].map(lambda ts:ts.strftime("%Y-%m-%d"))
df ['Date Locked']=df['Date Locked'].map(lambda ts:ts.strftime("%Y-%m-%d"))

df


#df.to_excel('Draft Invoice Report v1.xlsx')

import matplotlib.pyplot as plt
import xlsxwriter
import numpy as np
import pandas as pd
import pandas.io.sql
import os
import pdfkit
import pytz 
from datetime import datetime 
import base64
import pymssql 
import psycopg2 as pg
from jinja2 import Environment, FileSystemLoader

host        = '192.168.1.13'
database    = 'igu_live'
user        = 'sa'
password    = 'B1admin'

host2        = '192.168.1.13'
database2    = 'igu_webapp'
user2        = 'sa'
password2    = 'B1admin'

 
conn = pymssql.connect(host=host, user=user, password=password, database=database)
conn2 = pymssql.connect(host=host2, user=user2, password=password2, database=database2)

msg_sql3= """                
                select    
                        
                        c.cardcode  ,
                        c.cardname 'Partner Name',
                        d.groupname 'Group',
                        c.U_AR_Person , 
                      
                        
                        sum(a.BalScDeb -a.balsccred ) 'Total'
                from jdt1 a 
                inner join ojdt b on a.transid = b.transid 
                inner join ocrd c on a.ShortName = c.cardcode 
                inner join ocrg d on d.groupcode = c.groupcode 
                where 
                        a.account ='1130001' 
                        and a.BalScDeb -a.balsccred  <>0 
                        and convert(varchar,a.refdate,112)<= convert(varchar,getdate(),112)
                group by  
                        c.cardcode ,
                        d.groupname ,
                        c.cardname ,
                        c.U_AR_Person   
                
        """

pandas.options.display.float_format = '{:,.2f}'.format

#msg_sql = """ exec [dbo].[IGU_AGING_AR] '20200531','IGU' """
data3 = pandas.io.sql.read_sql(msg_sql3,conn)

msg_sql4= """
              select  a.id [Key],
                        c.Name itype,
                        a.TransNo,
                        coalesce(a.remarks,'') remarks,
                        convert(varchar,a.transdate,23) transdate,
                        coalesce(a.Collector ,'') Collector,
                        d.UserName,
                        a.amountDetailLc ,
                        a.CustomerCode cardcode ,
                        a.CustomerName ,
                        a.PayToCode ,
                        convert(varchar,b.InvoiceDate ,23) InvoiceDate,
                        b.InvoiceTypeName,
                        b.InvoiceRefNo NumAtCard,
                        b.InvoiceKwitansiNo ,
                        b.InvoiceFakturNo ,
                        b.InvoiceAmountTc 
                from Tx_PaymentIn a 
                    inner join Tx_PaymentIn_ArCorporate b on a.Id = b.Id
                    inner join Tm_User  d on a.ModifiedUser = d.Id
                    LEFT OUTER JOIN Ts_List c ON c.[Type] = 'PaymentInRefType' AND a.ReferenceType = c.Code
                where a.ReferenceType<>'Payment'
                and convert(varchar,A.transdate ,112) <= '20200905'
        """


data4 = pandas.io.sql.read_sql(msg_sql4,conn2)

 
#data
data3
data4
new_df = pandas.merge(data3, data4,  on=["cardcode"])

#new_df.to_excel("followup.xlsx")
#
# new_df[["cardcode"]]
#print(data)
#data3 =pandas.merge()

new_df.info()

#new_df.rename(columns = {'docnum':'No Invoice', 'NumAtCard' : 'No SO', 'docdate' : 'Invoice Date', 'cardcode' : 'Code','ShipToCode': 'Customer Name' ,'U_Kw_No': 'No kwitansi', 'U_AR_Person' : 'AR',
                    #'U_SlsEmpName': 'Sales', 'Balance':'Nominal','PymntGroup':'Term Of Payment', 'days':'TOPdays','U_locktimeout':'LockTimeout'  }, inplace = True)


dfTF = new_df[["cardcode", "Partner Name", "U_AR_Person", "itype", "TransNo", 'transdate', 'Collector', 'amountDetailLc', 'InvoiceDate','NumAtCard', 'InvoiceKwitansiNo','InvoiceAmountTc']]


dfTF

import pymssql as sapsql 

env = Environment(loader=FileSystemLoader('/data/jupyter/'))
#template = env.get_template("template.html")

datacompany = ({'host':'192.168.1.13' , 'dbname':'IGU_LIVE','user':'sa','password':'B1admin'},)

datalist5=[]
 
for company in datacompany:
    msg_sql5= """
                 exec DBO.IGU_INVOICELIST '20181101','20200904','',''
               
                """

    conn = sapsql.connect(host=company["host"] , user=company["user"] , password=company["password"], database=company["dbname"] )
    cursor = conn.cursor()     
    data5 = pandas.io.sql.read_sql(msg_sql5,conn)
    datalist5.append(data5)

df2 = pd.concat(datalist5)  
options = {
            'page-size': 'legal',
            'orientation': 'landscape',
            }
#html_out = template.render(template_vars)
textstring = ""
#test = df.pivot_table(index=["groupname",'cardname'],columns=["PymntGroup"],aggfunc=np.sum,  values=["Balance"],fill_value="0",margins=True)

#master = pd.DataFrame(df, columns = ['cardcode','cardname','groupname','U_AR_Person','LicTradNum','U_SlsEmpName','Balance','CreditLine','PymntGroup','U_locktimeout','U_IGU_LockBP',])

#for index,rows in data_ar.iterrows():
#    print(rows)

df2 = df2[['NumAtCard', 'balance']]

df2.info()

#test

df12 = pandas.merge(dfTF, df2, how = 'left', on=["NumAtCard"])

df12.info()

df12

env = Environment(loader=FileSystemLoader('/data/jupyter/'))
#template = env.get_template("template.html")

datacompany = ({'host':'192.168.1.13' , 'dbname':'IGU_LIVE','user':'sa','password':'B1admin'},)

datalist6=[]
 
for company in datacompany:
    msg_sql6= """
                   
                   select a.cardcode ,a.cardname ,d.groupname,a.U_AR_Person,c.U_SlsEmpName, a.Balance , a.CreditLine, a.U_locktimeout,
                            b.PymntGroup ,
                            b.ExtraDays days ,
                            b.ExtraMonth months

                    from OCRd a 
                        inner join octg b on a.GroupNum = b.GroupNum
                        inner join oslp c on a.slpcode = c.slpcode
                        inner join ocrg d on a.GroupCode = d.GroupCode
                    where a.cardtype='C'
                """

    conn = sapsql.connect(host=company["host"] , user=company["user"] , password=company["password"], database=company["dbname"] )
    cursor = conn.cursor()     
    data6 = pandas.io.sql.read_sql(msg_sql6,conn)
    datalist6.append(data6)

dfBPTF = pd.concat(datalist6)  
options = {
            'page-size': 'legal',
            'orientation': 'landscape',
            }
#html_out = template.render(template_vars)
textstring = ""
#test = df.pivot_table(index=["groupname",'cardname'],columns=["PymntGroup"],aggfunc=np.sum,  values=["Balance"],fill_value="0",margins=True)

dfBPTF = dfBPTF.fillna(0)

dfBPTF.info()
#for index,rows in data_ar.iterrows():
#    print(rows)

dfBPTF = dfBPTF[['cardcode', 'PymntGroup', 'days','U_locktimeout']]



#pdfkit.from_string(html_out,path_PDFfile1,options=options,css=cssfile) 
#dfBP.to_excel (r'bp310820.xlsx', index=True, header=True, merge_cells=False)

dfT = pandas.merge(df12, dfBPTF, how = 'left', on=["cardcode"])

dfT.info()

dfT

#new_df.rename(columns = {'docnum':'No Invoice', 'NumAtCard' : 'No SO', 'docdate' : 'Invoice Date', 'cardcode' : 'Code','ShipToCode': 'Customer Name' ,'U_Kw_No': 'No kwitansi', 'U_AR_Person' : 'AR',
                    #'U_SlsEmpName': 'Sales', 'Balance':'Nominal','PymntGroup':'Term Of Payment', 'days':'TOPdays','U_locktimeout':'LockTimeout'  }, inplace = True)
dfT.rename(columns = {'balance':'Nominal','U_locktimeout':'LockTimeout', 'TransNo':'TF No', 'transdate': 'TF date' , 'NumaAtCard' : 'number SO'}, inplace = True)

dfT ['Date Today']=pd.Timestamp("today")
dfT ['Date Today']=dfT['Date Today'].map(lambda ts:ts.strftime("%Y-%m-%d"))

dfT['TF date'] = pd.to_datetime(dfT['TF date'])
dfT['Date Today'] = pd.to_datetime(dfT['Date Today'])

dfT['TOP Jatuh Tempo'] = dfT['TF date'] + pd.to_timedelta(pd.np.ceil(dfT.days), unit="D")

dfT['H-7 JT days'] = (dfT['TOP Jatuh Tempo'] - dfT['Date Today'])

dfT['H-7 JT days'] = dfT['H-7 JT days'].dt.days

conditions = [
    (dfT['Nominal'] == 0),
    (dfT['H-7 JT days'] < 7) & (dfT['H-7 JT days'] >= 1) & (dfT['Nominal'] > 0),
    (dfT['H-7 JT days'] < 0) & (dfT['Nominal'] > 0),
    (dfT['H-7 JT days'] == 7) & (dfT['Nominal'] > 0),
    (dfT['H-7 JT days'] == 0) & (dfT['Nominal'] > 0),
       ]
choices = ['Lunas', 'Approaching JT', 'Overdue', "Today", "Jatuh Tempo"]

dfT['Alert H-7 JT'] = np.select(conditions, choices, default='')


dfT ['LockTimeout']=dfT['LockTimeout']*30

dfT['Date Locked'] = dfT['TF date'] + pd.to_timedelta(pd.np.ceil(dfT.LockTimeout), unit="D")

dfT['H-7 Locked days'] = (dfT['Date Locked'] - dfT['Date Today'])

dfT['H-7 Locked days'] = dfT['H-7 Locked days'].dt.days



conditions = [
    (dfT['Nominal'] == 0),
    (dfT['H-7 Locked days'] < 7) & (dfT['H-7 Locked days'] >= 1) & (dfT['Nominal'] > 0),
    (dfT['H-7 Locked days'] == 0) & (dfT['Nominal'] > 0),
    (dfT['H-7 Locked days'] == 7) & (dfT['Nominal'] > 0),
       ]
choices = ['Lunas', 'Approaching locked', 'Locked', "Today" ]

dfT['Alert H-7 Locked'] = np.select(conditions, choices, default='')

dfT['H Locked days'] = dfT['H-7 Locked days']

conditions = [
    (dfT['H Locked days'] == 0),
          ]
choices = ['Locked', ]

dfT['Alert H-Locked'] = np.select(conditions, choices, default='')

dfT.info()

dfT ['Date Today']=dfT['Date Today'].map(lambda ts:ts.strftime("%Y-%m-%d"))
dfT ['TF date']=dfT['TF date'].map(lambda ts:ts.strftime("%Y-%m-%d"))
dfT ['TOP Jatuh Tempo']=dfT['TOP Jatuh Tempo'].map(lambda ts:ts.strftime("%Y-%m-%d"))
dfT ['Date Locked']=dfT['Date Locked'].map(lambda ts:ts.strftime("%Y-%m-%d"))

#df2.to_excel("TF draft.xlsx")

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Invoice TF Report.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
#aging.to_excel(writer, sheet_name='ALL')
df.to_excel(writer, sheet_name='Invoice')
dfT.to_excel(writer, sheet_name = 'Tukar Faktur')

# Get the xlsxwriter objects from the dataframe writer object.
workbook  = writer.book
#worksheet1 = writer.sheets['ALL']
worksheet2 = writer.sheets['Invoice']
worksheet3 = writer.sheets['Tukar Faktur']

# Add some cell formats.
format1 = workbook.add_format({'num_format': '#,##0;[Red]-#,##0'})

# Set the column width and format. KAGA BISA AUTOFIT
#worksheet1.set_column('B:Z', None, format1)
worksheet2.set_column('J:V', None, format1)
worksheet3.set_column('I:Z', None, format1)

# Create a chart object.
#chart = workbook.add_chart({'type': 'column'})

# Configure the series of the chart from the dataframe data.
#chart.add_series({'values': '=sales!$B$4:$B$9'})

# Insert the chart into the worksheet.
#worksheet.insert_chart('C12', chart)

# Close the Pandas Excel writer and output the Excel file.
writer.save()

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  This is separate from the ipykernel package so we can avoid doing imports until


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24113 entries, 0 to 24112
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            24113 non-null  object 
 1   name          24113 non-null  object 
 2   docentry      24113 non-null  int64  
 3   docnum        24113 non-null  int64  
 4   NumAtCard     24044 non-null  object 
 5   docdate       24113 non-null  object 
 6   CANCELED      24113 non-null  object 
 7   cardcode      24113 non-null  object 
 8                 24113 non-null  object 
 9   ShipToCode    24113 non-null  object 
 10  doctype       24113 non-null  object 
 11  U_Kw_No       15368 non-null  object 
 12  U_IDU_FPajak  16590 non-null  object 
 13  U_Cust_PO_No  24113 non-null  object 
 14                24113 non-null  object 
 15                24113 non-null  object 
 16  usersign      24113 non-null  object 
 17  createdate    24113 non-null  object 
 18  doctime       24113 non-nu

C:\Users\user\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:104: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:123: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


<class 'pandas.core.frame.DataFrame'>
Int64Index: 12900 entries, 0 to 12899
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   cardcode           12900 non-null  object 
 1   Partner Name       12900 non-null  object 
 2   Group              12900 non-null  object 
 3   U_AR_Person        12900 non-null  object 
 4   Total              12900 non-null  float64
 5   Key                12900 non-null  int64  
 6   itype              12900 non-null  object 
 7   TransNo            12900 non-null  object 
 8   remarks            12900 non-null  object 
 9   transdate          12900 non-null  object 
 10  Collector          12900 non-null  object 
 11  UserName           12900 non-null  object 
 12  amountDetailLc     12900 non-null  float64
 13  CustomerName       12900 non-null  object 
 14  PayToCode          12900 non-null  object 
 15  InvoiceDate        12900 non-null  object 
 16  InvoiceTypeName    129

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:381: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:401: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
